In [31]:
import numpy as np
import pandas as pnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

testdata = pnd.read_csv('fashion-mnist_test.csv')
traindata = pnd.read_csv('fashion-mnist_train.csv')
testdata_pixel = testdata.drop('label', axis=1).to_numpy()
traindata_pixel = traindata.drop('label', axis=1).to_numpy()
label_train = traindata['label'].to_numpy()
label_test = testdata['label'].to_numpy()

cnn_test = testdata_pixel.reshape(-1,28,28) /255.0
cnn_train = traindata_pixel.reshape(-1,28,28) /255.0
cnn_test = np.expand_dims(cnn_test,axis=3)
cnn_train = np.expand_dims(cnn_train,axis=3)
cnn_label_test = tf.keras.utils.to_categorical(label_test)
cnn_label_train = tf.keras.utils.to_categorical(label_train)

We have now loaded and reshaped the data. The labels were converted into 10-dimensional to be compatible with the output of the softmax activation function and the dimension of the images was increased by one to show that there is only one color channel. 

In [19]:
model = Sequential([
  Conv2D(6, 5, input_shape=(28, 28, 1), activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(16, 5, activation="relu",padding="valid"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(120, 5, activation="relu",padding="valid"),
  Flatten(),
  Dense(84,activation="relu"),
  Dense(10, activation='softmax'),
])
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
model.fit(
  cnn_train,
  cnn_label_train,
  epochs=10,
  validation_data=(cnn_test, cnn_label_test)
)

Epoch 1/10
1875/1875 [==============================] - 39s 20ms/step - loss: 0.5333 - accuracy: 0.8044 - val_loss: 0.3922 - val_accuracy: 0.8604
Epoch 2/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.3548 - accuracy: 0.8697 - val_loss: 0.3392 - val_accuracy: 0.8758
Epoch 3/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.3057 - accuracy: 0.8889 - val_loss: 0.3204 - val_accuracy: 0.8802
Epoch 4/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.2758 - accuracy: 0.8992 - val_loss: 0.2710 - val_accuracy: 0.9014
Epoch 5/10
1875/1875 [==============================] - 47s 25ms/step - loss: 0.2543 - accuracy: 0.9051 - val_loss: 0.2764 - val_accuracy: 0.8956
Epoch 6/10
1875/1875 [==============================] - 55s 30ms/step - loss: 0.2380 - accuracy: 0.9119 - val_loss: 0.2752 - val_accuracy: 0.8982
Epoch 7/10
1875/1875 [==============================] - 46s 25ms/step - loss: 0.2225 - accuracy: 0.9168 - val_loss: 0.2720 -

The previous code used a modifyed LeNet-5 architecture with max pooling instead of mean pooling and ReLU activation

In [33]:
model = Sequential([
  Conv2D(32, 3, input_shape=(28, 28, 1), activation="relu",padding="same"),
  Conv2D(32, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Flatten(),
  Dense(256,activation="relu"),
  Dropout(0.15),
  Dense(256,activation="relu"),
  Dropout(0.15),
  Dense(10, activation='softmax'),
])
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
model.fit(
  cnn_train,
  cnn_label_train,
  epochs=5,
  validation_data=(cnn_test, cnn_label_test)
)

Epoch 1/5
1875/1875 [==============================] - 160s 82ms/step - loss: 0.5568 - accuracy: 0.7966 - val_loss: 0.3801 - val_accuracy: 0.8627
Epoch 2/5
1875/1875 [==============================] - 114s 61ms/step - loss: 0.3627 - accuracy: 0.8709 - val_loss: 0.3577 - val_accuracy: 0.8713
Epoch 3/5
1875/1875 [==============================] - 98s 52ms/step - loss: 0.3169 - accuracy: 0.8860 - val_loss: 0.2812 - val_accuracy: 0.8960
Epoch 4/5
1875/1875 [==============================] - 116s 62ms/step - loss: 0.2874 - accuracy: 0.8976 - val_loss: 0.2608 - val_accuracy: 0.9059
Epoch 5/5
1875/1875 [==============================] - 142s 76ms/step - loss: 0.2706 - accuracy: 0.9043 - val_loss: 0.2691 - val_accuracy: 0.9009


In [35]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_62 (Conv2D)          (None, 28, 28, 16)        160       
                                                                 
 conv2d_63 (Conv2D)          (None, 28, 28, 16)        2320      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 14, 14, 16)       0         
 g2D)                                                            
                                                                 
 batch_normalization_5 (Batc  (None, 14, 14, 16)       64        
 hNormalization)                                                 
                                                                 
 conv2d_64 (Conv2D)          (None, 14, 14, 16)        2320      
                                                                 
 conv2d_65 (Conv2D)          (None, 14, 14, 16)       